In [4]:
# import the necessary packages
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import imutils
import cv2
import matplotlib.pyplot as plt
import argparse

# load the image & resize it so as to speedup image processing, as well as make edge detection step more accurate
img = cv2.imread('omr_test_01.png')
ratio = img.shape[0]/500           # scaling the height to 500px
orig = img.copy()
img = imutils.resize(img, height=500)

# convert the image to grayscale, blur it & find its edges
gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray_img = cv2.GaussianBlur(gray_img, (5,5), 0)   # kernel_size = (5,5) +ve & odd std deviaton sigmaX=0, sigmaY if not defined equal to sigmaX
edged_img = cv2.Canny(gray_img, 75, 200)         # minVal=75, intensity grad below it are non edges & maxVal=200, above it are surely edges

# show the original image and edged image
print('Step 1: Edge detection')
cv2.imshow('Original', img)
cv2.imshow('Edged', edged_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# find contours in the edge map, then initialize the contour that corresponds to the document
cnts = cv2.findContours(edged_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None
# sort the contours in descending order
if len(cnts)>0:
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    # approximate the contours
    for i in cnts:
        peri = cv2.arcLength(i, True)
        epsilon = 0.02*peri
        approx = cv2.approxPolyDP(i, epsilon, True)
        # if approximated contour has 4 vertices then we can assume that we have found the paper
        if len(approx)==4:
            docCnt = approx
            break

print('Step 2: Find contours of paper')
cv2.drawContours(img, [docCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# apply the four point transform to obtain a top-down view of the original image
warped = four_point_transform(orig, docCnt.reshape(4, 2) * ratio)
# convert the warped image to grayscale, then threshold it to give it 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = cv2.threshold(warped,0,255,
                      cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
warped = (warped > T).astype("uint8") * 255
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)

Step 1: Edge detection
Step 2: Find contours of paper
STEP 3: Apply perspective transform


-1